In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
from sqlalchemy import create_engine
from pysandag.database import get_connection_string

db_connection_string = get_connection_string('..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)

### 1. Get output of sr13

### get output of sr13:  unit change grouped by jurisdiction

In [ ]:
sr13_sql_match = '''
    SELECT c.jurisdiction, c.jurisdiction_id, c.yr_from, c.yr_to, c.hu_change
    FROM (SELECT a.jurisdiction, a.jurisdiction_id, a.yr_id AS yr_from, b.yr_id AS yr_to
    ,CASE WHEN b.hu - a.hu > 0 THEN b.hu - a.hu ELSE 0 END AS hu_change
    FROM (SELECT y.jurisdiction, y.jurisdiction_id, x.yr_id, sum(x.units) AS hu
    FROM [demographic_warehouse].[fact].[housing] AS x
    inner join [demographic_warehouse].[dim].[mgra_denormalize] AS y ON x.mgra_id = y.mgra_id
    WHERE x.datasource_id = 13 and x.yr_id in (2015, 2020, 2025, 2030, 2035, 2040, 2045, 2050)
    GROUP BY y.jurisdiction, y.jurisdiction_id, x.yr_id) AS a
    inner join 
    (SELECT y.jurisdiction,y.jurisdiction_id,x.yr_id, sum(x.units) AS hu
    FROM [demographic_warehouse].[fact].[housing] AS x
    inner join [demographic_warehouse].[dim].[mgra_denormalize] AS y ON x.mgra_id = y.mgra_id
    WHERE x.datasource_id = 13 and x.yr_id in (2015, 2020, 2025, 2030, 2035, 2040, 2045, 2050)
    GROUP BY y.jurisdiction, y.jurisdiction_id, x.yr_id) AS b
    ON a.jurisdiction_id = b.jurisdiction_id and a.yr_id = b.yr_id-5) AS c
    ORDER BY yr_from, jurisdiction_id
'''

In [ ]:
sr13_units_by_jur = pd.read_sql(sr13_sql_match,mssql_engine)

### capacity from [sr13_final].[capacity] (includes sched dev)

In [ ]:
sr13_cap_by_jur_sql = '''
    SELECT y.City,sum([cap_hs]) AS cap
    FROM [regional_forecast].[sr13_final].[capacity] x
    inner join [regional_forecast].[sr13_final].[mgra13] AS y ON x.mgra = y.mgra
    WHERE x.scenario = 0 and x.increment in (2012) 
    GROUP BY y.City, x.increment
    ORDER BY y.City'''

In [ ]:
sr13_cap_by_jur =  pd.read_sql(sr13_cap_by_jur_sql,mssql_engine,index_col='City')

### set increment as 'yr_to'

In [ ]:
sr13_units_by_jur['increment'] = sr13_units_by_jur['yr_to']

In [ ]:
sr13_units_by_jur = sr13_units_by_jur.drop(['yr_from', 'yr_to'],axis =1)

In [ ]:
# sr13_units_by_jur.head()

### join sr13 unit change with capacity

In [ ]:
units_by_jur_inc = sr13_units_by_jur.set_index('jurisdiction_id').join(sr13_cap_by_jur)

In [ ]:
units_by_jur_inc.head()

### add starting year as 2012 with unit change as 0 (for plotting)

In [ ]:
start_year = units_by_jur_inc.loc[units_by_jur_inc.increment==2025].copy()

In [ ]:
# start_year

In [ ]:
start_year['increment'] = 2012
start_year['hu_change'] = 0

In [ ]:
units_by_jur_inc = pd.concat([start_year,units_by_jur_inc])

In [ ]:
units_by_jur_inc.head()

### pivot for plotting so jurisdictions are columns and increments are rows with new units

In [ ]:
units_by_jur_pivot = units_by_jur_inc.pivot\
(index='increment', columns='jurisdiction', values='hu_change').\
reset_index().rename_axis(None, axis=1)
units_by_jur_pivot.fillna(0,inplace=True)
units_by_jur_pivot.set_index('increment',inplace=True)

### pivot for plotting so jurisdictions are columns and increments are rows with capacity

In [ ]:
cap_by_jur_pivot = units_by_jur_inc.pivot\
(index='increment', columns='jurisdiction', values='cap').\
reset_index().rename_axis(None, axis=1)
cap_by_jur_pivot.fillna(0,inplace=True)
cap_by_jur_pivot.set_index('increment',inplace=True)

### calculate cumulative sum of units added by jurisdiction

In [ ]:
units_by_jur_pivot = units_by_jur_pivot.cumsum()

### plot results

#### for each jurisdiction join dataframe of new units and capacity

#### calculate remaining capacity by subtracting new units from starting capacity

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(nrows=10, ncols=2)
for j, jur in enumerate(units_by_jur_inc.jurisdiction.unique().tolist()):
    df_units_added = units_by_jur_pivot[[jur]]
    df_capacity = cap_by_jur_pivot[[jur]]
    df_plot = df_units_added.join(df_capacity, lsuffix='_new', rsuffix='_capacity')
    hs_column = jur + '_new'
    cap_column = jur + '_capacity'
    cap_plot_column = jur + '_cap'
    df_plot[cap_plot_column] = df_plot[cap_column] - df_plot[hs_column]
    del  df_plot[cap_column]
    df_plot.plot(style='.-',ax=axes.flat[j],figsize=(10,32))
   # df_plot.plot(kind='bar',ax=axes.flat[j],figsize=(10,32)))

In [ ]:
df_plot.plot(kind='bar')